In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np
import os


numpoints = 1001
albedo_map = np.loadtxt('albedo_map1_n'+str(numpoints)+'.txt',delimiter=',')

voronoi_grid = np.loadtxt('voronoi_grid_n'+str(numpoints),delimiter=',')
plot_facets_vor = np.load('plot_facets_n'+str(numpoints)+'.npy',allow_pickle=',')
phi_grid,theta_grid,area_grid = voronoi_grid.transpose()

In [2]:
'''
Parameters
'''
res = len(phi_grid)
rho = 6.371e6
R = 1.496e11

res_alpha = 3
res_beta = 3

alpha = np.linspace(0/4*np.pi,2/2*np.pi,res_alpha)
beta = np.linspace(0/4*np.pi,1/2*np.pi,res_beta)

day = 24
year = 365*day
omega_day = 2*np.pi/day
omega_year = 2*np.pi/year

N_ave = 187

f_curve = np.loadtxt('f_curves/edge_t1024.txt',delimiter=',')
time_res = len(f_curve)

'''
Albedo-map
'''
A = np.zeros((res,res_alpha*res_beta))
f_arrays = np.zeros((time_res,res_alpha*res_beta))


'''
Euler rotation matrices
'''
def positive(arg):
    return(arg+abs(arg))/2

def y_rotation(angle):
    Y = np.array([[np.cos(angle),0,np.sin(angle)],[0,1,0],
                  [-np.sin(angle),0,np.cos(angle)]])
    return Y

def z_rotation(angle):
    Z = np.array([[np.cos(angle),-np.sin(angle),0],
                  [np.sin(angle),np.cos(angle),0],[0,0,1]])
    return Z

'''
Initialize transfromation matrices (edge-on, face-on)
'''

T = np.zeros((time_res,res,res_alpha*res_beta))
T_pinv = np.zeros((res,time_res,res_alpha*res_beta))

'''
Observer
'''
o_vec = np.array([1,0,0])

In [12]:
%%time
'''
Compute matrix elements
'''
for k in range(res_alpha):
    R_equinox = z_rotation(alpha[k])
    for l in range(res_beta):
        R_tilt = y_rotation(beta[l])
        R_axial = np.matmul(R_equinox,R_tilt)
        for i in range(len(time_array)):
            t = time_array[i]
            r_vec = np.array([-np.cos(omega_year*t),-np.sin(omega_year*t),0])
            R_daily = z_rotation(omega_day*t)
            daily_rotation = np.matmul(R_axial,R_daily)

            for j in range(res):
                phi = phi_grid[j]
                theta = theta_grid[j]
                s_vec = np.array([np.cos(phi)*np.sin(theta),
                                  np.sin(phi)*np.sin(theta),np.cos(theta)])
                s_vec_rotated = np.matmul(daily_rotation,s_vec)

                r_s = np.dot(r_vec,s_vec_rotated)
                s_o = np.dot(s_vec_rotated,o_vec)

                illuminated = positive(r_s)
                visible = positive(s_o)


                T[i][j][k*res_beta+l] = illuminated*visible*np.sin(theta)*area_grid[j] 

        T_ = T[:,:,k*res_beta+l]
        T_pinv_ = np.linalg.pinv(T_,rcond = 10e-6)
        T_pinv[:,:,k*res_beta+l] = T_pinv_
        A_ = np.matmul(T_pinv_,f_curve)
        A[:,k*res_beta+l] = A_
        f_arrays[:,k*res_beta+l] = np.matmul(T_,A_)*rho**2/(R**2*np.pi)
        print('(alpha, beta) ',k,l)

filename = 'transfer_matrices/multiT_a'+str(res_alpha)+'_b'+str(res_beta)+'.npy'
os.makedirs(os.path.dirname(filename), exist_ok=True)
np.save(filename,T,allow_pickle=True)

f_curve_matrix = np.tile(f_curve,(res_alpha*res_beta,1)).transpose()
f_diff = np.square(f_curve_matrix-f_arrays)
square_diff = np.sum(f_diff,axis = 0)
minima = np.where(square_diff == square_diff.min())[0]

alpha_ = np.floor(minima/res_beta).astype(int)
beta_ = minima % res_beta

print('alpha = ',np.degrees(alpha[alpha_]))
print('beta = ',np.degrees(beta[beta_]))

(alpha, beta)  0 0
(alpha, beta)  0 1
(alpha, beta)  0 2
(alpha, beta)  1 0
(alpha, beta)  1 1
(alpha, beta)  1 2
(alpha, beta)  2 0
(alpha, beta)  2 1
(alpha, beta)  2 2
alpha =  [90.]
beta =  [90.]
Wall time: 2min 45s
